___
## Cargue de librerias

In [ ]:
!pip install unidecode

In [ ]:
import pandas as pd
import numpy as np
from math import pi
import matplotlib.pyplot as plt
from datetime import datetime
import unidecode
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import seaborn as sns
from matplotlib.patches import Patch

___
## Instalación del entorno de GitHub

In [ ]:
# Instalar git en Colab si no está instalado
!apt-get install git

In [ ]:
# Clonar el repositorio de GitHub
!git clone https://github.com/niconinov/DatasetAnalitica.git

___
## Descarga de las bases de datos en el Notebook

In [ ]:
names =['Camaras', 'Hurtos', 'Recuperacion1', 'Recuperacion2', 'Recuperacion3', 'Estratos barrios', 'Estratos comunas']

In [ ]:
dfs = {}

for i in names:
  dfs[i] = pd.read_csv(f'/content/DatasetAnalitica/{i}.csv')

In [ ]:
df_cam = pd.DataFrame(dfs['Camaras'])
df_hurto = pd.DataFrame(dfs['Hurtos'].iloc[:, 1:])
df_rec1 = pd.DataFrame(dfs['Recuperacion1'].iloc[:, 1:])
df_rec2 = pd.DataFrame(dfs['Recuperacion2'].iloc[:, 1:])
df_rec3 = pd.DataFrame(dfs['Recuperacion3'].iloc[:, 1:])
df_estratos_barrio = pd.DataFrame(dfs['Estratos barrios'].iloc[:, 1:])
df_estratos_comuna = pd.DataFrame(dfs['Estratos comunas'].iloc[:, 1:])

In [ ]:
df_hurto

In [ ]:
df_estratos_comuna

In [ ]:
df_estratos_barrio

---
# **Gráficos**

### Diagrama de torta

In [ ]:
conteo = df_hurto['codigo_comuna'].value_counts()
conteo = dict(conteo)
conteo = dict(sorted(conteo.items()))
conteo

In [ ]:
explo = [0,0,0,0,0,0,0,0,0,0.2,0,0,0,0,0,0]

fig, ax = plt.subplots(figsize=(10, 10))
ax.pie(conteo.values(), labels=conteo.keys(), colors = ['#D3D3D3', 'aquamarine', '#1E90FF', 'teal'], explode = explo, autopct='%1.1f%%', startangle=180)
ax.axis('equal')    # De este modo la figura tendrá forma circular
fig.show()

___
### Linea de tiempo

#### Agrupación por mes hurtos totales

In [ ]:
grouped_df2 = df_hurto.groupby(['Año', 'Mes']).size().reset_index(name='count')
grouped_df2['Mes'] = grouped_df2['Mes'].replace({
                                              'Ene':1, 'Feb':2, 'Mar':3, 'Abr':4,
                                              'May':5, 'Jun':6, 'Jul':7, 'Ago':8,
                                              'Sep':9, 'Oct':10, 'Nov':11, 'Dic':12
                                              })
grouped_df2['Fecha'] = grouped_df2.apply(lambda row: f"{row['Año']}-{row['Mes']}", axis=1)
grouped_df2['Fecha'] = pd.to_datetime(grouped_df2['Fecha'], format='%Y-%m')
grouped_df2 = grouped_df2.sort_values(by='Fecha')
grouped_df2 = grouped_df2.drop(columns=['Año', 'Mes'])
grouped_df2 = grouped_df2.iloc[:, [1, 0]]
grouped_df2 = grouped_df2.reset_index(drop=True)
print(grouped_df2)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(grouped_df2['Fecha'], grouped_df2['count'], marker = 'o');
plt.title('Evolución de los hurtos en el tiempo', fontdict = {'fontsize':20, 'fontweight':'bold'})
plt.ylabel('Cantidad de hurtos', fontdict = {'fontsize':15})
plt.grid(axis = 'y', color = 'green', linestyle = 'dashed')
fig.show(warn=False)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=grouped_df2['Fecha'], y=grouped_df2['count'], mode='lines+markers'))
fig.update_traces(line=dict(color='#1B676B', width=1),
                  marker=dict(size=8, symbol='diamond-wide'))
fig.update_layout(
    title='Evolución de los hurtos en el tiempo', title_font=dict(size=30, color='black',  family="Arial Black"),
    yaxis_title='Cantidad de hurtos',
    showlegend=False,
    width=1500,
    height=500,
    plot_bgcolor='white'
)
# Agregar grilla a al eje y
fig.update_yaxes(showgrid=True, gridcolor='gray')
# Dar fotomato al eje x
fig.update_xaxes(
    dtick="M6",
    tickformat="%b\n%Y",
    ticklabelmode="period")

fig.show()

#### Agrupación por mes hurtos por comuna

In [ ]:
df_hurto['codigo_comuna'] = df_hurto['codigo_comuna'].astype(int)

In [ ]:
grouped_dfv = df_hurto.groupby(['Año', 'Mes', 'codigo_comuna']).size().reset_index(name='count')
grouped_dfv['Mes'] = grouped_dfv['Mes'].replace({
                                              'Ene':1, 'Feb':2, 'Mar':3, 'Abr':4,
                                              'May':5, 'Jun':6, 'Jul':7, 'Ago':8,
                                              'Sep':9, 'Oct':10, 'Nov':11, 'Dic':12
                                              })
grouped_dfv['Fecha'] = grouped_dfv.apply(lambda row: f"{row['Año']}-{row['Mes']}", axis=1)
grouped_dfv['Fecha'] = pd.to_datetime(grouped_dfv['Fecha'])
grouped_dfv = grouped_dfv.sort_values(by='Fecha')
grouped_dfv = grouped_dfv.drop(columns=['Año', 'Mes'])
grouped_dfv = grouped_dfv.reset_index(drop=True)
print(grouped_dfv)

In [ ]:
comunas = {}
for i in range(1, 17):
    comuna_name = 'comuna' + ' ' + str(i)
    comunas[comuna_name] = grouped_dfv[grouped_dfv['codigo_comuna'] == i].reset_index(drop=True).sort_values(by='Fecha')

In [ ]:
# fig, ax = plt.subplots(figsize=(20, 6))
# for i in range(1,17):
#     ax.plot(comunas['comuna' + str(i)]['Fecha'], comunas['comuna' + str(i)]['count'],marker = 'o', label = 'Comuna ' + str(i));
# ax.legend(loc='upper left')
# plt.title('Evolución de los hurtos por comuna en el tiempo', fontdict = {'fontsize':15, 'fontweight':'bold'})
# fig.show(warn=False)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(comunas['comuna 10']['Fecha'], comunas['comuna 10']['count'],marker = 'o', label = 'Comuna 10');
ax.legend(loc='upper left')
plt.grid(axis = 'y', color = 'green', linestyle = 'dashed')
plt.title('Evolución de los hurtos por comuna en el tiempo', fontdict = {'fontsize':15, 'fontweight':'bold'})
fig.show(warn=False)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=comunas['comuna 1']['Fecha'], y=comunas['comuna 1']['count'], mode='lines+markers'))
fig.update_traces(line=dict(color='#1B676B', width=1),
                  marker=dict(size=8, symbol='diamond-wide'))
fig.update_layout(
    title='Evolución de los hurtos en el tiempo, en la comuna 1', title_font=dict(size=30, color='black',  family="Arial Black"),
    yaxis_title='Cantidad de hurtos',
    showlegend=False,
    width=1500,
    height=500,
    plot_bgcolor='white'
)
# Agregar grilla a al eje y
fig.update_yaxes(showgrid=True, gridcolor='gray')
# Dar fotomato al eje x
fig.update_xaxes(
    dtick="M6",
    tickformat="%b\n%Y",
    ticklabelmode="period")

fig.show()

In [ ]:
conteo = df_hurto['nombre_comuna'].value_counts()
conteo

In [ ]:
contcam =df_cam['comuna'].value_counts()
contcam

#### Graficos individuales de las 4 comunas con mas camaras

In [ ]:
comunas_top4 = ['comuna 14', 'comuna 10', 'comuna 5', 'comuna 15']

# Graficar las comunas
for i in comunas_top4:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=comunas[i]['Fecha'], y=comunas[i]['count'], mode='lines+markers'))
    fig.update_traces(line=dict(color='#1B676B', width=1),
                      marker=dict(size=8, symbol='diamond-wide'))
    fig.update_layout(
        title=f'Evolución de los hurtos en el tiempo, en la {i}', title_font=dict(size=30, color='black',  family="Arial Black"),
        yaxis_title='Cantidad de hurtos',
        showlegend=False,
        width=1500,
        height=500,
        plot_bgcolor='white'
    )
    # Agregar grilla a al eje y
    fig.update_yaxes(showgrid=True, gridcolor='gray')
    # Dar fotomato al eje x
    fig.update_xaxes(
        dtick="M6",
        tickformat="%b\n%Y",
        ticklabelmode="period")

    fig.show()

#### Graficos agrupados de las 4 comunas con mas camaras

In [ ]:
comunas_top4 = ['comuna 14', 'comuna 10', 'comuna 5', 'comuna 15']

# Definir colores y marcadores
colors = ['#1B676B', '#519548', '#88C425', '#BEF202']
markers = ['diamond-wide', 'circle', 'square', 'triangle-up']

# Crear figura
fig = go.Figure()

# Agregar trazas para cada comuna
for i, comuna in enumerate(comunas_top4):
    fig.add_trace(go.Scatter(x=comunas[comuna]['Fecha'], y=comunas[comuna]['count'],
                             mode='lines+markers',
                             line=dict(color=colors[i], width=1),
                             marker=dict(size=8, symbol=markers[i]),
                             name=comuna
                             ))  # Asignar nombre a la traza

# Configurar layout
fig.update_layout(
    title='Evolución de los hurtos en el tiempo, en las comunas con mas camaras del SIMM',
    title_font=dict(size=30, color='black',  family="Arial Black"),
    yaxis_title='Cantidad de hurtos',
    showlegend=True,
    width=1500,
    height=500,
    plot_bgcolor='white'
)

# Agregar grilla a al eje y
fig.update_yaxes(showgrid=True, gridcolor='gray')
# Dar formato al eje x
fig.update_xaxes(
    dtick="M6",
    tickformat="%b\n%Y",
    ticklabelmode="period")

fig.show()

#### Graficos agrupados de forma individual de las 4 comunas con mas camaras

In [ ]:
from plotly.subplots import make_subplots
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "xy"}, {"type": "xy"}],
           [{"type": "xy"}, {"type": "xy"}]],
)

fig.add_trace(go.Scatter(x=comunas['comuna 14']['Fecha'], y=comunas['comuna 14']['count'],
                         mode='lines+markers',
                         line=dict(color='#1B676B', width=1),
                         marker=dict(size=8, symbol='diamond'),
                         name='El Poblado'),
              row=1, col=1)
fig.add_trace(go.Scatter(x=comunas['comuna 10']['Fecha'], y=comunas['comuna 10']['count'],
                         mode='lines+markers',
                         line=dict(color='#519548', width=1),
                         marker=dict(size=8, symbol='diamond'),
                         name = 'Candelaria'),
              row=1, col=2)
fig.add_trace(go.Scatter(x=comunas['comuna 5']['Fecha'], y=comunas['comuna 5']['count'],
                         mode='lines+markers',
                         line=dict(color='#88C425', width=1),
                         marker=dict(size=8, symbol='diamond'),
                         name = 'Castilla'),
              row=2, col=1)
fig.add_trace(go.Scatter(x=comunas['comuna 15']['Fecha'], y=comunas['comuna 15']['count'],
                         mode='lines+markers',
                         line=dict(color='#BEF202', width=1),
                         marker=dict(size=8, symbol='diamond'),
                         name = 'Guayabal'),
              row=2, col=2)

fig.update_layout(
    title='Evolución de los hurtos en el tiempo, en las comunas con mas camaras del SIMM',
    title_font=dict(size=30, color='black',  family="Arial Black"),
    title_x=0.5,
    height=900,
    showlegend=True,
    plot_bgcolor='white')

fig.update_yaxes(title_text="Cantidad de hurtos", row=1, col=1)
fig.update_yaxes(title_text="Cantidad de hurtos", row=1, col=2)
fig.update_yaxes(title_text="Cantidad de hurtos", row=2, col=1)
fig.update_yaxes(title_text="Cantidad de hurtos", row=2, col=2)

fig.show()

___
### Mapa de calor

Cantidad de hurtos segun la hora y el dia de la semana

In [ ]:
# Crear una columna para el día de la semana a partir de la fecha
hmap = df_hurto.groupby(['Año', 'Mes', 'Día', 'Hora']).size().reset_index(name='count')

hmap['Mes'] = hmap['Mes'].map({
    'Ene': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Ago': 8,
    'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dic': 12
})

hmap['Fecha'] = pd.to_datetime(hmap.apply(lambda row: f"{row['Año']}-{row['Mes']}-{row['Día']}", axis=1))
hmap['Día_de_la_semana'] = hmap['Fecha'].dt.day_name()

# Crear rangos de una hora
hmap['Hora_rango'] = pd.cut(hmap['Hora'], bins=range(0, 2500, 100), right=False, labels=[f'{str(i).zfill(4)}-{str(i+100).zfill(4)}' for i in range(0, 2400, 100)])

# Crear una tabla dinámica con los días de la semana como filas y las horas como columnas
heatmap_data = hmap.pivot_table(index='Día_de_la_semana', columns='Hora_rango', aggfunc='size', fill_value=0)

# Ordenar los días de la semana
ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
hmap['Mes'] = hmap['Mes'].map({
    'Monday':'Lunes', 'Tuesday':'Martes',
    'Wednesday':'Miercoles', 'Thursday':'Jueves',
    'Friday':'Viernes', 'Saturday':'Sabado',
    'Sunday':'Domingo'
})
heatmap_data = heatmap_data.reindex(ordered_days)

fig = px.imshow(heatmap_data,
               color_continuous_scale='Reds',
               title='Número de hurtos por hora y día de la semana',
               labels=dict(x="Hora del día", y="Día de la semana"),
               text_auto=True)
fig.show()

---
### Diagrama de barras

#### Cantidad vs dia de la semana

In [ ]:
diagra = df_hurto.groupby(['Año', 'Mes', 'Día']).size().reset_index(name='count')

diagra['Mes'] = diagra['Mes'].map({
    'Ene': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Ago': 8,
    'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dic': 12
})

diagra['Fecha'] = pd.to_datetime(diagra.apply(lambda row: f"{row['Año']}-{row['Mes']}-{row['Día']}", axis=1))
diagra = diagra.drop(columns=['Año', 'Mes', 'Día']).reset_index(drop=True)
diagra = diagra.sort_values(by='Fecha')

diagra['Día'] = diagra['Fecha'].dt.day_name()
diagramadias = diagra.groupby('Día')['count'].sum().reset_index()

diagramadias['Día'] = diagramadias['Día'].map({
    'Monday': 'Lunes', 'Tuesday': 'Martes', 'Wednesday': 'Miércoles',
    'Thursday': 'Jueves', 'Friday': 'Viernes', 'Saturday': 'Sábado',
    'Sunday': 'Domingo'
})
diagramadias["Día"] = pd.Categorical(diagramadias["Día"], categories=[
    'Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo'
], ordered=True)
diagramadias = diagramadias.sort_values(by='Día')
diagramadias = diagramadias.reset_index(drop=True)

diagramadias


In [ ]:
# Definir una lista de colores para cada barra
colores = ['#00ff7f', '#34ff8d', '#62ff95', '#8cffaa', '#adffc0', '#cbffd5', '#e5ffea']

fig = px.bar(diagramadias, x='Día', y='count', color='Día', color_discrete_sequence=colores)
fig.show()

#### Cantidad vs hombre-mujer

Diagrama de barras agrupadas que muestre según el genero la cantidad de hurtos que ocurren cada día de la semana, buscando una correlación temporal con los días de la semana para inducir el diagrama de hurtos por fin de semana


In [ ]:
diagrami = df_hurto.groupby(['Año', 'Mes', 'Día', 'sexo']).size().reset_index(name='count')

diagrami['Mes'] = diagrami['Mes'].map({
    'Ene': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Ago': 8,
    'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dic': 12
})

diagrami['Fecha'] = pd.to_datetime(diagrami.apply(lambda row: f"{row['Año']}-{row['Mes']}-{row['Día']}", axis=1))
diagrami = diagrami.drop(columns=['Año', 'Mes', 'Día']).reset_index(drop=True)
diagrami = diagrami.sort_values(by='Fecha')

diagrami['Día'] = diagrami['Fecha'].dt.day_name()
diagrami

In [ ]:
diagramasexo = diagrami.groupby(['Día', "sexo"])["count"].sum().unstack().reset_index()
diagramasexo = diagramasexo[['Día', 'Hombre', 'Mujer']]
diagramasexo['Día'] = diagramasexo['Día'].map({
    'Monday': 'Lunes', 'Tuesday': 'Martes', 'Wednesday': 'Miércoles',
    'Thursday': 'Jueves', 'Friday': 'Viernes', 'Saturday': 'Sábado',
    'Sunday': 'Domingo'
})
diagramasexo["Día"] = pd.Categorical(diagramasexo["Día"], categories=[
    'Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo'
], ordered=True)
diagramasexo = diagramasexo.sort_values(by='Día')
diagramasexo = diagramasexo.reset_index(drop=True)
diagramasexo

In [ ]:
colorM = ['#d851d6', '#e45ce1', '#ef67ec', '#f484f0', '#f89ef3', '#fbb7f6', '#fdd0f9']
colorH = ['#baa8ff', '#9f8dff', '#8072ff', '#5757ff', '#4d50f7', '#414aef', '#3543e7']

fig = go.Figure()

fig.add_trace(go.Bar(name='Mujer', x=diagramasexo['Día'], y=diagramasexo['Mujer'], marker_color=colorM))
fig.add_trace(go.Bar(name='Hombre', x=diagramasexo['Día'], y=diagramasexo['Hombre'], marker_color=colorH))

fig.show()

#### Cantidad por día de la semana según comuna

In [ ]:
diagrami = df_hurto.groupby(['Año', 'Mes', 'Día', 'nombre_comuna']).size().reset_index(name='count')

diagrami['Mes'] = diagrami['Mes'].map({
    'Ene': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Ago': 8,
    'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dic': 12
})

diagrami['Fecha'] = pd.to_datetime(diagrami.apply(lambda row: f"{row['Año']}-{row['Mes']}-{row['Día']}", axis=1))
diagrami = diagrami.drop(columns=['Año', 'Mes', 'Día']).reset_index(drop=True)
diagrami = diagrami.sort_values(by='Fecha')

diagrami['Día'] = diagrami['Fecha'].dt.day_name()
diagrami

In [ ]:
diagramacomuna = diagrami.groupby(['Día', 'nombre_comuna'])['count'].sum().unstack().reset_index()
diagramacomuna

#### Cantidad vs hora

In [ ]:
# Crear rangos de una hora
rangos= pd.cut(df_hurto['Hora'], range(0, 2500, 100))
# Contar hurtos por rango de hora
hurtos_por_rango = rangos.value_counts()
# Ordenar hurtos por rango de hora
hurtos_por_rango = hurtos_por_rango.sort_index()
hurtos_por_rango

In [ ]:
s = pd.DataFrame(hurtos_por_rango)
s = s.reset_index().rename(columns={'index': 'indices', 'count': 'Hurtos'})

In [ ]:
s.plot(kind='bar', x='Hora', y='Hurtos', color= 'g')
plt.title('Hurtos por rango horario')
plt.xlabel('Rango horario')
plt.ylabel('Cantidad de hurtos')
plt.show()

In [ ]:
s['Hora'] = s['Hora'].astype(str)

fig = px.bar(s, x='Hora', y='Hurtos', color_discrete_sequence=['#00FF7F'])
fig.update_layout(title='Hurtos por rango horario',
                  xaxis_title='Rango horario',
                  yaxis_title='Cantidad de hurtos',
                  plot_bgcolor='white')
# Agregar grilla a los ejes
fig.update_yaxes(showgrid=True, gridcolor='gray')
fig.show()

#### Modalidad vs comuna

In [ ]:
df_hurto['modalidad'].unique()

In [ ]:
barras = df_hurto.groupby(['codigo_comuna', 'modalidad']).size().reset_index(name='count')
barras

In [ ]:
df_rec1

In [ ]:
import pandas as pd

df_rec1

# Agrupar por 'año' y 'codigo_comuna', y sumar la columna 'cantidad'
df_grouped = df_rec1.groupby(['año', 'codigo_comuna']).sum().reset_index()

print(df_grouped)

In [ ]:
def agrupar_por_año_y_comuna(df_rec1):
    # Agrupar por 'año' y 'codigo_comuna', y sumar la columna 'cantidad'
    df_grouped = df_rec1.groupby(['año', 'codigo_comuna']).sum().reset_index()
    return df_grouped

In [ ]:
agrupar_por_año_y_comuna(df_rec1)

In [ ]:
# import pandas as pd

# # Crear un dataframe de ejemplo

# df_rec1

# def filtrar_y_agrupas(df, codigo_comuna=None, año=None):
#     # Filtrar por codigo_comuna si se especifica
#     if codigo_comuna is not None:
#         df = df[df['codigo_comuna'] == codigo_comuna]

#     # Filtrar por año si se especifica
#     if año is not None:
#         df = df[df['año'] == año]

#     # Agrupar por 'año' y 'codigo_comuna', y sumar la columna 'cantidad'
#     df_grouped = df.groupby(['año', 'codigo_comuna']).sum().reset_index()

#     return df_grouped

# # Usar la función para obtener datos filtrados y agrupados
# # Ejemplo: Filtrar por codigo_comuna = 1 y año = 2022
# result = filtrar_y_agrupas(df_rec1, 11, 2020)
# print(result)


In [ ]:
df_rec2.head()

In [ ]:
df_rec3.head()

#### Hurtos por estrato

In [ ]:
df_estratos_comuna = df_estratos_comuna.rename(columns={'Nombre Comuna': 'nombre_comuna'})
df_estratos_comuna['nombre_comuna'] = df_estratos_comuna['nombre_comuna'].replace({'MANRRIQUE': 'MANRIQUE'})

In [ ]:
# Realizar la unión de ambos DataFrames según la comuna
df_hurto_estratos_comunas = df_hurto[['codigo_comuna', 'nombre_comuna']].copy()
df_hurto_estratos_comunas = pd.merge(df_hurto_estratos_comunas, df_estratos_comuna, on='nombre_comuna', how='left')
df_hurto_estratos_comunas = df_hurto_estratos_comunas.drop(columns='Comuna')
df_hurto_estratos_comunas

In [ ]:
hurtos_por_estrato = df_hurto_estratos_comunas['Estrato'].value_counts().reset_index().rename(columns={'index': 'Estrato', 'count': 'count'})
hurtos_por_estrato['Estrato'] = hurtos_por_estrato['Estrato'].replace({2:'Estrato 2', 3:'Estrato 3', 4:'Estrato 4',
                                                                       5:'Estrato 5', 6:'Estrato 6'})
hurtos_por_estrato

In [ ]:
colores = ['#1B676B', '#519548', '#88C425', '#BEF202', '#ADFFC0']

fig = px.bar(hurtos_por_estrato, x='Estrato', y='count', color='Estrato', color_discrete_sequence=colores)
fig.show()

### Grafico de radar

In [ ]:
# Contar cuántas veces se repiten los valores únicos en la columna 'codigo_comuna'
conteo_comunas = df_hurto['codigo_comuna'].value_counts().reset_index()
# Renombrar las columnas del nuevo DataFrame
conteo_comunas.columns = ['codigo_comuna', 'hurtos']
# Fusionar conteo_comunas y df_rec21 por la columna 'codigo_comuna'
spider = pd.merge(conteo_comunas, df_rec2, on='codigo_comuna')
# Ordenear de forma ascendente segun el código de la comuna
spider = spider.sort_values(by='codigo_comuna').reset_index(drop=True)
# Modificar los valores de la columna codigo_comuna
spider['codigo_comuna'] = 'Com. ' + spider['codigo_comuna'].astype(str)
spider['Relación'] = spider['cantidad_recuperaciones']/spider['hurtos']

spider = spider.rename(columns={'hurtos': 'Hurto', 'cantidad_recuperaciones': 'Recuperaciones'})

spider

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatterpolar(
        r=spider['Hurto'],
        theta=spider['codigo_comuna'],
        fill='toself',
        name='Hurtos',
        marker=dict(color='#8cff57')
    )
)
fig.add_trace(
    go.Scatterpolar(
        r=spider['Recuperaciones'],
        theta=spider['codigo_comuna'],
        fill='toself',
        name='Recuperaciones',
        marker=dict(color='#f680e6')
    )
)

fig.update_layout(
    polar=dict(
        angularaxis=dict(
            tickmode='array',
            tickvals=spider['codigo_comuna'],
            ticktext=spider['nombre_comuna']
        )
    )
)

fig.show()

### Mapa relación hurtos-comunas

In [ ]:
df_combinado = df_hurto[['latitud', 'longitud', 'nombre_comuna']].copy()
df_combinado = pd.merge(df_combinado, df_estratos_comuna, on='nombre_comuna', how='left')
df_combinado = df_combinado.drop(columns='Comuna')
df_combinado.dropna(subset=['latitud', 'longitud'], inplace=True)
df_combinado

In [ ]:
#coordenadas promedio de las comunas para el mapa de hurtos y recuperaciones
promedio_coords = df_combinado.groupby('nombre_comuna').mean().reset_index()
cant_hurtos = df_combinado['nombre_comuna'].value_counts().reset_index()
cant_hurtos.columns = ['nombre_comuna', 'cantidad_hurtos']
promedio_coords = promedio_coords.merge(cant_hurtos, on='nombre_comuna', how='left')
promedio_coords['Estrato'] = promedio_coords['Estrato'].replace({2.0:'Estrato 2', 3.0:'Estrato 3', 4.0:'Estrato 4',
                                                                  5.0:'Estrato 5', 6.0:'Estrato 6'})
promedio_coords

In [ ]:
# Crear el mapa interactivo utilizando Plotly Express
fig = px.scatter_mapbox(promedio_coords, lat='latitud', lon='longitud', hover_name='nombre_comuna',
                        color='Estrato', size='cantidad_hurtos',
                        color_continuous_scale=px.colors.sequential.Viridis,
                        size_max=50, zoom=10)

# Actualizar el estilo del mapa
fig.update_layout(mapbox_style='open-street-map')

# Mostrar el mapa
fig.show()

### Conteo combinado de hurtos y cámaras por comuna con diferentes colores

In [ ]:
hurtos_por_comuna = df_hurto['nombre_comuna'].value_counts().reset_index().rename(columns={'index': 'comuna', 'count': 'count'})
df_cam = df_cam.rename(columns={'comuna': 'nombre_comuna'})
camaras_por_comuna = df_cam['nombre_comuna'].value_counts().reset_index().rename(columns={'index': 'comuna', 'count': 'count'})

In [ ]:
#DataFrames
hurtos_por_comuna
camaras_por_comuna

# Acceder a columnas específicas
nombres_por_comuna = hurtos_por_comuna['nombre_comuna']
conteos_por_comuna = hurtos_por_comuna['count']

nombres_por_comuna = camaras_por_comuna['nombre_comuna']
conteos_por_comuna = camaras_por_comuna['count']

# Combinar los DataFrames
combined_df = pd.concat([hurtos_por_comuna, camaras_por_comuna], ignore_index=True)

# Identificar los nombres comunes y únicos
common_names = set(hurtos_por_comuna['nombre_comuna']).intersection(set(camaras_por_comuna['nombre_comuna']))
unique_names = set(combined_df['nombre_comuna']) - common_names

# Asignar colores
combined_df['color'] = combined_df['nombre_comuna'].apply(lambda x: '#008B8B' if x in common_names else '#7FFFD4')

# Crear el gráfico de barras
plt.figure(figsize=(16, 6))
bars = plt.bar(combined_df['nombre_comuna'], combined_df['count'], color=combined_df['color'])

# Crear los parches para la leyenda
legend_elements = [Patch(facecolor='#008B8B', label='Comunas con cámaras'),
                   Patch(facecolor='#7FFFD4', label='Comunas sin cámaras')]

# Girar los nombres del eje x
plt.xticks(rotation=45, ha='right')

# Añadir la leyenda
plt.legend(handles=legend_elements, loc='upper right')
plt.xlabel('Nombre Comuna')
plt.ylabel('Cantidad de hurtos')
plt.title('Conteo combinado de hurtos y cámaras por comuna con diferentes colores')
plt.show()

### Mapa cantidad recuperaciones

In [ ]:
df_combo = pd.merge(df_rec2[['nombre_comuna', 'cantidad_recuperaciones']], promedio_coords[['nombre_comuna', 'latitud', 'longitud']], on='nombre_comuna')
df_combo

In [ ]:
# Crear el mapa interactivo utilizando Plotly Express
fig = px.scatter_mapbox(df_combo, lat='latitud', lon='longitud', hover_name='nombre_comuna',
                        color='nombre_comuna', size='cantidad_recuperaciones',
                        color_continuous_scale=px.colors.sequential.Viridis,
                        size_max=50, zoom=10)

# Actualizar el estilo del mapa
fig.update_layout(mapbox_style='open-street-map')

# Mostrar el mapa
fig.show()

___
# **DashBoard**

___
## Instalación de paquetes

In [ ]:
!pip install -q jupyter-dash
!pip install -q dash_bootstrap_templates

___
## Cargue de librerias

In [ ]:
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

## Código Dash

In [ ]:
app = dash.Dash(__name__)

# Define el estilo de las pestañas
tab_style = {
    'borderBottom': '1px solid #5f7060',
    'padding': '6px',
    'backgroundColor': '#86d99b',  # Color de fondo de las pestañas
}

# Define el estilo de la pestaña activa
tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'fontWeight': 'bold',
    'backgroundColor': '#34b356',  # Color de fondo de la pestaña activa
    'color': '#000000'  # Color del texto de la pestaña activa
}

# Función para la línea de tiempo
def linea_tiempo(df):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Fecha'], y=df['count'], mode='lines+markers'))
    fig.update_traces(line=dict(color='#1B676B', width=1), marker=dict(size=8, symbol='diamond-wide'))
    fig.update_layout(
        yaxis_title='Cantidad de hurtos', showlegend=False, width=1250, height=400, plot_bgcolor='white')
    # Agregar grilla a al eje y
    fig.update_yaxes(showgrid=True, gridcolor='gray')
    # Dar formato al eje x
    fig.update_xaxes(dtick="M6", tickformat="%b\n%Y", ticklabelmode="period", tickangle=-45, tickfont=dict(size=10, family="Arial"))
    return fig

# Graficar las comunas
def linea_tiempo_com(comunas, comuna):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=comunas[comuna]['Fecha'], y=comunas[comuna]['count'], mode='lines+markers'))
    fig.update_traces(line=dict(color='#5f9497', width=1), marker=dict(size=8, symbol='diamond-wide'))
    fig.update_layout(
        title=f'Evolución de los hurtos en el tiempo, en la {comuna}', title_font=dict(size=15, color='black',  family="Arial Black"),
        yaxis_title='Cantidad de hurtos', showlegend=False, width=1250, height=400, plot_bgcolor='white')
    # Agregar grilla a al eje y
    fig.update_yaxes(showgrid=True, gridcolor='gray')
    # Dar formato al eje x
    fig.update_xaxes(dtick="M6", tickformat="%b\n%Y", ticklabelmode="period", tickangle=-45, tickfont=dict(size=10, family="Arial"))
    return fig

dropdown_options = [{'label': k, 'value': k} for k in comunas.keys()]

# Grafico de barras, hurtos por dia
def bar_plot(df):
  colores = ['#00ff7f', '#34ff8d', '#62ff95', '#8cffaa', '#adffc0', '#cbffd5', '#e5ffea']
  fig = px.bar(df, x='Día', y='count', color='Día', color_discrete_sequence=colores, labels={'count': 'Cantidad de hurtos'})
  return fig

# Heatmap
import pandas as pd
import plotly.express as px

def heatmap(df):
    # Create a new column for the day of the week based on the date
    hmap = df.groupby(['Año', 'Mes', 'Día', 'Hora']).size().reset_index(name='count')
    hmap['Mes'] = hmap['Mes'].map({'Ene': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4,'May': 5, 'Jun': 6, 'Jul': 7, 'Ago': 8,'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dic': 12})
    hmap['Fecha'] = pd.to_datetime(hmap.apply(lambda row: f"{row['Año']}-{row['Mes']}-{row['Día']}", axis=1))
    hmap['Día_de_la_semana'] = hmap['Fecha'].dt.day_name()
    # Create ranges of one hour
    hmap['Hora_rango'] = pd.cut(hmap['Hora'], bins=range(0, 2500, 100), right=False, labels=[f'{str(i).zfill(4)}-{str(i+100).zfill(4)}' for i in range(0, 2400, 100)])
    # Create a pivot table with the days of the week as rows and hours as columns
    heatmap_data = hmap.pivot_table(index='Día_de_la_semana', columns='Hora_rango', aggfunc='size', fill_value=0)
    # Sort the days of the week
    ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    hmap['Día_de_la_semana'] = hmap['Día_de_la_semana'].map({'Monday':'Lunes','Tuesday':'Martes','Wednesday':'Miercoles','Thursday':'Jueves','Friday':'Viernes','Saturday':'Sabado','Sunday':'Domingo'})
    heatmap_data = heatmap_data.reindex(ordered_days)
    fig = px.imshow(heatmap_data,
                   color_continuous_scale='Greens',labels=dict(x="Hora del día", y="Día de la semana"),text_auto=True)
    return fig

#barras hurto por estrato
def barras_estrato(df):
    colores = ['#1B676B', '#519548', '#88C425', '#BEF202', '#ADFFC0']
    estrato_orden = ['Estrato 2', 'Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 6']
    fig = px.bar(hurtos_por_estrato, x='Estrato', y='count', color='Estrato', color_discrete_sequence=colores, labels={'count': 'Cantidad de hurtos'}, category_orders={"Estrato": estrato_orden})
    return fig

# Crear el gráfico de barras utilizando Plotly
def crear_grafico_combinado(hurtos_por_comuna, camaras_por_comuna):
    combined_df = pd.concat([hurtos_por_comuna, camaras_por_comuna], ignore_index=True)
    common_names = set(hurtos_por_comuna['nombre_comuna']).intersection(set(camaras_por_comuna['nombre_comuna']))
    combined_df['color'] = combined_df['nombre_comuna'].apply(lambda x: '#7FFFD4' if x in common_names else '#008B8B')

    fig = go.Figure()

    for color, label in zip(['#7FFFD4', '#008B8B'], ['Comunas con cámaras', 'Comunas sin cámaras']):
        df_filtered = combined_df[combined_df['color'] == color]
        fig.add_trace(go.Bar(
            x=df_filtered['nombre_comuna'],
            y=df_filtered['count'],
            name=label,
            marker_color=color
        ))

    fig.update_layout(
        #title='Conteo combinado de hurtos y cámaras por comuna',
        xaxis_title='Nombre Comuna',
        yaxis_title='Cantidad de hurtos',
        barmode='stack',
        legend_title_text='Tipo de Comuna',
        xaxis={'categoryorder':'total descending'}
    )

    return fig

def create_spider_plot(spider):
    fig = go.Figure()

    fig.add_trace(
        go.Scatterpolar(
            r=spider['Hurto'],
            theta=spider['codigo_comuna'],
            fill='toself',
            name='Hurtos',
            marker=dict(color='#88C425')
        )
    )
    fig.add_trace(
        go.Scatterpolar(
            r=spider['Recuperaciones'],
            theta=spider['codigo_comuna'],
            fill='toself',
            name='Recuperaciones',
            marker=dict(color='#1B676B')
        )
    )

    fig.update_layout(
        polar=dict(
            angularaxis=dict(
                tickmode='array',
                tickvals=spider['codigo_comuna'],
                ticktext=spider['nombre_comuna']
            )
        )
    )

    return fig

def mapa_estrato(df):
    # Definir el orden de los estratos
    estrato_orden = ['Estrato 2', 'Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 6']

    # Asegurarse de que los valores de la columna 'Estrato' sean exactamente iguales a los valores en 'estrato_orden'
    df['Estrato'] = df['Estrato'].astype(str)

    # Convertir la columna 'Estrato' a categoría con el orden especificado
    df['Estrato'] = pd.Categorical(df['Estrato'], categories=estrato_orden, ordered=True)

    # Ordenar el DataFrame por la columna 'Estrato'
    df = df.sort_values('Estrato')

    # Definir los colores específicos para cada estrato
    colores = {
        'Estrato 2': '#1B676B',
        'Estrato 3': '#519548',
        'Estrato 4': '#88C425',
        'Estrato 5': '#BEF202',
        'Estrato 6': '#ADFFC0'
    }

    fig = px.scatter_mapbox(df, lat='latitud', lon='longitud', hover_name='nombre_comuna',
                            color='Estrato', size='cantidad_hurtos',
                            color_discrete_map=colores,
                            size_max=50, zoom=10)
    fig.update_layout(mapbox_style='open-street-map')
    return fig

def mapa_recuperaciones(df_combo):
    # Crear una función para categorizar las cantidades de recuperaciones
    def categorizar_recuperaciones(cantidad):
        if cantidad < 1500:
            return 'Baja recuperación (Menos de 1500)'
        elif 1500 <= cantidad < 3000:
            return 'Mediana recuperación (Entre 1500 y 3000)'
        else:
            return 'Alta recuperación (Más de 3000)'

    # Aplicar la función a la columna 'cantidad_recuperaciones' para crear la columna 'categoria'
    df_combo['Categoría'] = df_combo['cantidad_recuperaciones'].apply(categorizar_recuperaciones)

    # Crear el mapa con las categorías de colores
    fig = px.scatter_mapbox(df_combo, lat='latitud', lon='longitud', hover_name='nombre_comuna',
                            color='Categoría', size='cantidad_recuperaciones',
                            color_discrete_map={
                                'Baja recuperación (Menos de 1500)': '#1B676B',
                                'Mediana recuperación (Entre 1500 y 3000)': '#519548',
                                'Alta recuperación (Más de 3000)': '#88C425'
                            },
                            size_max=50, zoom=10)
    fig.update_layout(mapbox_style='open-street-map')
    return fig

#from plotly.subplots import make_subplots
#import plotly.graph_objects as go

def crear_grafico_hurtos(comunas):
    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{"type": "xy"}, {"type": "xy"}],
               [{"type": "xy"}, {"type": "xy"}]],
    )

    fig.add_trace(go.Scatter(x=comunas['comuna 14']['Fecha'], y=comunas['comuna 14']['count'],
                             mode='lines+markers',
                             line=dict(color='#1B676B', width=1),
                             marker=dict(size=5, symbol='diamond'),
                             name='El Poblado'),
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=comunas['comuna 10']['Fecha'], y=comunas['comuna 10']['count'],
                             mode='lines+markers',
                             line=dict(color='#519548', width=1),
                             marker=dict(size=5, symbol='diamond'),
                             name='Candelaria'),
                  row=1, col=2)
    fig.add_trace(go.Scatter(x=comunas['comuna 5']['Fecha'], y=comunas['comuna 5']['count'],
                             mode='lines+markers',
                             line=dict(color='#88C425', width=1),
                             marker=dict(size=5, symbol='diamond'),
                             name='Castilla'),
                  row=2, col=1)
    fig.add_trace(go.Scatter(x=comunas['comuna 15']['Fecha'], y=comunas['comuna 15']['count'],
                             mode='lines+markers',
                             line=dict(color='#BEF202', width=1),
                             marker=dict(size=5, symbol='diamond'),
                             name='Guayabal'),
                  row=2, col=2)

    fig.update_layout(
        #title='Evolución de los hurtos en el tiempo, en las comunas con mas camaras del SIMM',
        title_font=dict(size=30, color='black', family="Arial Black"),
        title_x=0.5,
        height=900,
        showlegend=True,
        plot_bgcolor='white')

    fig.update_yaxes(title_text="Cantidad de hurtos", row=1, col=1)
    fig.update_yaxes(title_text="Cantidad de hurtos", row=1, col=2)
    fig.update_yaxes(title_text="Cantidad de hurtos", row=2, col=1)
    fig.update_yaxes(title_text="Cantidad de hurtos", row=2, col=2)

    return fig

app.layout = html.Div([
    dcc.Tabs(id='tabs', value='graph1',
             children=[
                 dcc.Tab(label='Presentación', style=tab_style, selected_style=tab_selected_style, children=[
                     html.Hr(),
                     html.H1(html.Div('Análisis de Hurtos de Motocicletas en Medellín: Patrones, Estratificación y Vigilancia', style={'text-align': 'center', 'font-weight': 'bold'})),
                     html.Hr(),
                     html.H2('Autores', style={'text-align': 'center', 'font-weight': 'bold'}),
                     html.Ul([
                         html.Li('Valentina Jimenez'),
                         html.Li('Nicolás Niño'),
                         html.Li('David Díaz')
                         ]),
                     html.Hr(),
                     html.Img(src='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRx0VcgOF6vInjePYzBMCThsefZhOrM8OVV9GkdVWjg6GzJiPzu0zI_hwyA1_S78Uwf7mc&usqp=CAU',
                              style={'width': '200px', 'display': 'block', 'margin': 'auto'}),
                     html.Hr()
                    ]),

                 dcc.Tab(label='Introducción', style=tab_style, selected_style=tab_selected_style, children=[
                     html.Hr(),
                     html.H4('Breve descripción del estudio y objetivos', style={'text-align': 'center', 'font-weight': 'bold'}),
                     html.P('Este estudio tiene como objetivo analizar los patrones de hurtos de motocicletas en la ciudad de Medellín, explorando la relación entre la\n'
                     'estratificación socioeconómica y la efectividad de la vigilancia mediante cámaras SIMM.'),
                     html.Hr(),
                     html.H4('Metodología empleada y fuentes de datos utilizadas', style={'text-align': 'center', 'font-weight': 'bold'}),
                     html.P('Para este análisis se utilizaron datos de hurtos de motocicletas proporcionados por la policía de Medellín y datos abiertos en repostorios como Medata, así como información sobre la\n'
                     'estratificación socioeconómica de las zonas de la ciudad. Se emplearon técnicas de análisis espacial y temporal para identificar patrones y relaciones entre las variables.'),
                     html.Hr(),
                     html.H4('Fuentes de información', style={'text-align': 'center', 'font-weight': 'bold'}),
                     html.Ul([
                         html.Li('Datos abietos sobre hurtos de motocicletas'),
                         html.Li('Datos de las cámaras SIMM.'),
                         html.Li('Información sociodemográfica de la ciudad.')
                         ]),
                     html.Hr(),
                     html.H4('Resumen de las preguntas clave que se quieren responder', style={'text-align': 'center', 'font-weight': 'bold'}),
                     html.Ul([
                         html.Li('¿Existen patrones espaciales y temporales que se repiten en el tiempo?'),
                         html.Li('¿Existe relación entre la estratificación del área donde se registra el hurto con la cantidad de hurtos registrados?'),
                         html.Li('¿Hay baja frecuencia de hurtos en las zonas monitorizadas con cámaras SIMM?'),
                         html.Li('¿Existe relación entre el área de recuperación de una motocicleta con la estratificación de esta área?')
                         ]),
                     html.Hr(),
                    ]),

                 dcc.Tab(label='Visión General de los Hurtos', style=tab_style, selected_style=tab_selected_style, children=[
                     html.Div([
                         html.Hr(),
                         html.H4("Distribución de Hurtos por Día de la Semana", style={'textAlign': 'center'}),
                         dcc.Graph(id='graph3', figure=bar_plot(diagramadias)),
                         html.Hr(),
                         html.H4("Evolución de Hurtos en el Valle de Aburrá", style={'textAlign': 'center'}),
                         dcc.Graph(id='graph1', figure=linea_tiempo(grouped_df2)),
                         html.Hr(),
                         html.H4("Evolución de Hurtos en el Valle de Aburrá, segun la comuna", style={'textAlign': 'center'}),
                         html.P("Seleccione una comuna para ver su evolución de los hurtos en el tiempo."),
                         dcc.Dropdown(
                             id='comuna-disp-dropdown',
                             options=dropdown_options,
                             value=[0]),
                         dcc.Graph(id='graph2'),
                         ]),
                     ]),

                 dcc.Tab(label='Patrones Temporales', style=tab_style, selected_style=tab_selected_style, children=[
                     html.Div([
                         html.H4("Mapa de Calor de Hurtos por Día y Hora", style={'textAlign': 'center'}),
                         dcc.Graph(id='graph4', figure=heatmap(df_hurto)),
                             ]),
                     ]),

                 dcc.Tab(label='Análisis por Estrato', style=tab_style, selected_style=tab_selected_style, children=[
                    html.Div([
                        html.Hr(),
                        html.H4('Diagrama de barras de la cantidad de hurto respecto al estrato de la comuna', style={'textAlign': 'center'}),
                        dcc.Graph(id='graph5', figure=barras_estrato(hurtos_por_estrato)),
                        html.Hr(),
                        html.H4('Mapa de la cantidad de hurtos por comuna según estrato', style={'textAlign': 'center'}),
                        dcc.Graph(id='graph7', figure=mapa_estrato(promedio_coords))])
                     ]),
                dcc.Tab(label='Eficacia de Cámaras SIMM', style=tab_style, selected_style=tab_selected_style, children=[
                    html.Div([
                        html.Hr(),
                        html.H4('Conteo de hurtos por tipo de comuna', style={'textAlign': 'center'}),
                        dcc.Graph(id='graph8', figure=crear_grafico_combinado(hurtos_por_comuna, camaras_por_comuna)),
                        html.Hr(),
                        html.H4('Evolución de los hurtos en el tiempo, en las comunas con mas camaras del SIMM', style={'textAlign': 'center'}),
                        dcc.Graph(id='graph9', figure=crear_grafico_hurtos(comunas))])
                     ]),
                 dcc.Tab(label='Recuperación de Motocicletas', style=tab_style, selected_style=tab_selected_style, children=[
                     html.Div([
                        html.Hr(),
                        html.H4('Grafico de radar que contrasta la cantidad de hurtos y recuperaciones por comuna.', style={'textAlign': 'center'}),
                        dcc.Graph(id='graph10', figure=create_spider_plot(spider)),
                        html.Hr(),
                        html.Div([html.H4('Recuperaciones de Motocicletas Robadas por Comuna en Medellín', style={'textAlign': 'center'}),
                                  dash_table.DataTable(id='table', columns = [{"name": "Código Comuna", "id": spider.columns[0]},
                                                                    {"name": "Nombre Comuna", "id": spider.columns[2]},
                                                                    {"name": "Hurtos", "id": spider.columns[1]},
                                                                    {"name": "Recuperacionees", "id": spider.columns[3]},
                                                                    {"name": "Tasa de recuperación", "id": spider.columns[4]},],
                                             data=spider.to_dict('records'), style_data={'width': '8px', 'height': '3px'}),
                                  ], style={'margin': '150px'}),
                        html.Hr(),
                        html.H4('Mapa de la cantidad de recuperaciones por comunas', style={'textAlign': 'center'}),
                        dcc.Graph(id='graph11', figure=mapa_recuperaciones(df_combo))])
                     ]),
                 dcc.Tab(label='Conclusiones', style=tab_style, selected_style=tab_selected_style, children=[
                     html.Hr(),
                     html.H4('Patrones Temporales:', style={'text-align': 'center', 'font-weight': 'bold'}),
                     html.Ul([
                         html.Li('Los hurtos son más frecuentes los lunes, martes y miércoles, con el miércoles siendo el día de mayor incidencia.\n'
                         'Los hurtos disminuyen hacia el fin de semana, siendo el domingo el día con menos hurtos.'),
                         html.Li('Horas del día: Los hurtos son más comunes durante la madrugada y las primeras horas de la mañana (5-7 am) y nuevamente\n'
                         'en la noche (6-10 pm). Esto puede estar relacionado con la movilidad laboral y la menor vigilancia durante estas horas.\n')
                     ]),
                     html.Hr(),
                     html.H4('Tendencias a lo largo del tiempo:', style={'text-align': 'center', 'font-weight': 'bold'}),
                     html.Ul([
                         html.Li('Los hurtos han mostrado fluctuaciones significativas en los últimos 20 años, con periodos de alta y baja incidencia.'),
                         html.Li('Se observó un pico en los hurtos entre 2002-2004, una disminución entre 2005-2010, un nuevo aumento entre 2011-2013, y\n'
                         'fluctuaciones moderadas entre 2014-2019. La pandemia de COVID-19 en 2020 causó una caída notable, seguida de un aumento en 2021-2022\n')
                     ]),
                     html.Hr(),
                     html.H4('Análisis por Estrato:', style={'text-align': 'center', 'font-weight': 'bold'}),
                     html.Ul([
                         html.Li('En el análisis del hurto de motos en el Valle de Aburrá, se destaca que la mayor cantidad de estos delitos ocurre en el estrato 3.\n'
                         'Este estrato incluye predominantemente las comunas Aranjuez, Buenos Aires y Belén, donde se registra una alta incidencia de robos de motocicletas.')
                     ]),
                     html.Hr(),
                     html.H4('Eficacia de las cámaras SIMM:', style={'text-align': 'center', 'font-weight': 'bold'}),
                     html.Ul([
                         html.Li('El análisis de la eficacia de las cámaras del Sistema Inteligente de Movilidad (SIMM) en el Valle de Aburrá revela que estas herramientas son cruciales para el monitoreo de las calles en diversas comunas. Estas cámaras permiten identificar vehículos en tránsito y realizar rastreos de actos delictivos, lo que debería contribuir a una mayor seguridad.\n'
                         'Sin embargo, no todas las comunas están equipadas con estas cámaras. También se observa que algunas comunas que sí cuentan con estas cámaras, como La Candelaria y Aranjuez, presentan una mayor incidencia de hurtos de motos.')
                     ]),
                     html.Hr(),
                     html.H4('Recuperación de motocicletas:', style={'text-align': 'center', 'font-weight': 'bold'}),
                     html.Ul([
                         html.Li('El análisis de la recuperación de motos hurtadas en el Valle de Aburrá muestra una tendencia preocupante: a medida que aumenta el número de hurtos, disminuye la cantidad de motos recuperadas. Esta disminución en las recuperaciones se puede suponer, gran parte al desarrollo y sofisticación de las modalidades delictivas, lo que dificulta significativamente la labor de las autoridades para capturar a los delincuentes y recuperar los vehículos.\n'
                         )
                     ]),
                 ]),
    ]),
])


@app.callback(
    Output("graph2", "figure"),
    [Input("comuna-disp-dropdown", "value")]
)
def update_graph2(comuna):
    # Llama a la función linea_tiempo_com con el diccionario comunas y el valor seleccionado en el dropdown
    fig = linea_tiempo_com(comunas, comuna)
    return fig

@app.callback(
    Output("spider-plot", "figure"),
    [Input("tabs", "value")]
)

def update_spider_plot(tab):
    if tab == 'Recuperación de Motocicletas':
        fig = create_spider_plot(spider)
        return fig
    return go.Figure()

app.run_server(jupyter_mode="external")